# Statistical Models in Chapter 6

The transition to a quadratic model -- along with the interactions -- proved to be somewhat intractable. So, I ended up using a Bayesian Multilevel Model to estimate these effects. It's actually amazingly simple, and some of the GGPLOT functions make for a more effective graphical presentation. 

* Multilevel model using BRMS, as opposed to 7 individual regression models corresponding to each year. This requires fitting one model, as opposed to 7.
* The approach makes it easy to compare different models.
* Use posterior predictions to generate point predictions/confidence regions.

The raw data are stored in the github drive entitled $\texttt{pooled.auth.rda}$. All the "source" files are also in the github repo. 

The workflow is as follows

* Estimate all regression models, save the output. This is achieved in this notebook. It creates a number of files -- it does take a good amount of time to estimate everything. Much of it is probably overkill. The traceplots and other diagnostics all look fine. I'm not sure it takes the number of iterations specified to reach convergence. It's useful to run everything. I then process the models in a separate notebook.
* Post-process these regression models, forming the notebook called "post_process.ipynb." Figures and graphics are shown in this notebook.

I followed a multilevel approach. 

$$
pr(y_{i,t}| x_{auth, i, t}; x^2_{auth, i, t}; z_{k, i, t}) =\beta_{0,k} + \beta_{1,t} x_{auth, i, t} +  \beta_{2,t} x_{auth, i, t}^2 + \sum_{k=1}^K \beta_k z_{{k,i,t} +  \epsilon_{t} +  \epsilon_{i,t}
$$

where $K$ denotes the number of control variables, $t$ cross-sectional year, and $i$, the $i$ th respondent (nested within the $t$ ANES survey). Wide priors were used for all the parameters, and the correlations between slopes and intercepts were always estimated (with the exception of the random intercept only model). Logistic regression was used for voting, multinomial logit for party identification, and linear regression for the affective polarization analysis. Throughout the analysis, I also specify interactions between authoritarianism and education. The output is then saved to three lists, analyzed in the *post_process* notebook.


In [ ]:
##Load dependencies, packages, data, etc ##
# More transformations and data recodes.....
rm(list = ls())
library(brms)
library(tidyverse)
library(ggplot2)
library(tictoc)
library(modelr)
library(tidybayes)
library(dplyr)
library(cowplot)
#### Some helper functions
source("/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/configurations/configurations.r")
data_location = "/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/clean_data/"
setwd("/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/Chapters/Chapter8/Chapter6")
#### Load cleaned data
load("/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/clean_data/pooled.auth.rda")  ### Just work from this data; everything should be here, recoded.
##### persistent ggtheme
ggtheme = theme(
        plot.title=element_text(face="bold",hjust=-.08,vjust=0,colour="#3C3C3C",size=12),
        axis.text.x=element_text(size=9,colour="#535353",face="bold"),
        axis.text.y=element_text(size=9,colour="#535353",face="bold"),
        axis.title.y=element_text(size=11,colour="#535353",face="bold",vjust=1.5),
        axis.ticks=element_blank(),
        panel.grid.major=element_line(colour="#D0D0D0",size=.25),
        panel.background=element_rect(fill="white")) 
####
data$authoritarianism_2 = data$authoritarianism^2 # Attempted with orthogonal contrasts, but the incompatibility with other stuff made it a challenge

In [2]:
### I create smaller, more tractable versions of the data rather than operating on the full data frame, data.
tmp_dat = data[,c("vote", "authoritarianism", 
                 "female", "age", "college", "income",
                 "jewish", "catholic", "other", "year")] %>% na.omit() %>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism) 

I ran these models in $\texttt{brms}$, a useful package to run a bunch of different types of regression models in a bayesian framework. BRMS ends up just creating $\texttt{stan}$ code that is more efficient than I could do on my own. There are three models

* $\texttt{fit0a}$ is a random intercept model. Intercepts vary by year. Everything else is fixed. 
* $\texttt{fit0b}$ is a random intercept and random slope model. Intercepts vary by year, as does the effect of authoritarianism. Everything else is fixed. 
* $\texttt{fit1}$ is a random intercept and random slope model. Intercepts vary by year, as does the effect of authoritarianism. Authoritarianism, however, has a quadratic effect. Everything else is fixed (and linear).
* $\texttt{education}$ is a random intercept and random slope model. Intercepts vary by year, as does the effect of authoritarianism. Authoritarianism, however, has a quadratic effect. I specficy an interaction between authoritarianism, and education; as well as authoritarianism$^2$ and education. The code is below. I ended up just saving the output in a list so I wouldn't need to estimate the models every time I work on the project (and we can save this for replication).



This is the random intercept model. I ran it over 3000 iterations, 3 chains. 

In [3]:

fit0a <- brm(vote~ female + age + college + income + jewish + 
                 catholic + other + authoritarianism +  
                 (1|year), 
                 family = bernoulli(link = "logit"),
                 data = tmp_dat, 
                 chains = 3, cores = 6, seed = 1234, 
                 iter = 1000)

Compiling Stan program...

Start sampling

Warning message:
"There were 1 divergent transitions after warmup. See
http://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them."
Warning message:
"Examine the pairs() plot to diagnose sampling problems
"


This model allows for random intercepts and a random slope for authoritarianism. BRMS also estimates the covariance between the two random effects, which is important in these models.

In [ ]:
fit0b <- brm(vote~ female + age + college + income + jewish + 
                 catholic + other + authoritarianism +  
                 (1+authoritarianism|year), 
                 family = bernoulli(link = "logit"),
                 data = tmp_dat, 
                 chains = 3, cores = 8, seed = 1234, 
                 iter = 1000)

Ultimately, it's the model below that I'll use. In the post_processing notebook, I compare the model fit across these models. This analysis supports a quadratic specification.

In [ ]:
fit1 <- brm(vote~ female + age + college + income + jewish + 
                 catholic + other + authoritarianism + authoritarianism_2 + 
                 (1+authoritarianism + authoritarianism_2|year), 
                 family = bernoulli(link = "logit"),
                 data = tmp_dat, 
                 chains = 3, cores = 8, seed = 1234, 
                 iter = 1000)


Finally, this model specifies an interaction with education. The models are saved in a list entitled, "vote_model.rda." Nothing takes all that long to estimate -- but it is annoying to wait -- so I saved the output and load for subsequent analysis.

In [ ]:
education <- brm(vote~ female + age + college + income + jewish + 
                  catholic + other + authoritarianism + authoritarianism_2 + 
                  authoritarianism:college + authoritarianism_2:college + 
                  (1+ authoritarianism + authoritarianism_2 |year), 
                  family = bernoulli(link = "logit"),
                  data = tmp_dat, 
                  chains = 3, 
                  cores = options(mc.cores = parallel::detectCores()), 
                  seed = 1234, 
                  iter = 2000)
vote_models = list(fit0a, fit0b, fit1, education)
save(vote_models, file = "vote_model.rda")

## Party Identification Models


Below I fit two models to a mulltinomial logit predicting party identification. One includes the effect of authortarianism, and authoritarianism-squared; the second includes an interaction with education. I save these in a list entitled, party_models.rda.

In [2]:
data$party3      = recode(data$pid*6 + 1, `1` = 1, `2` = 1, `3` = 2 ,`4` = 2, `5` = 2, `6` = 3, `7` = 3) %>% suppressWarnings()
data$republican  = recode(data$pid*6 + 1, `1` = 0, `2` = 0, `3` = 0 ,`4` = 0, `5` = 0, `6` = 1, `7` = 1) %>% suppressWarnings()
data$democrat    = recode(data$pid*6 + 1, `1` = 1, `2` = 1, `3` = 0 ,`4` = 0, `5` = 0, `6` = 0, `7` = 0) %>% suppressWarnings()
data$independent = recode(data$pid*6 + 1, `1` = 0, `2` = 0, `3` = 1 ,`4` = 1, `5` = 1, `6` = 0, `7` = 0) %>% suppressWarnings()
data$feeling.repc = zero.one(data$feeling.repc) %>% as.numeric
data$feeling.demc = zero.one(data$feeling.demc) %>% as.numeric
data$feeling.rep  = zero.one(data$feeling.rep) %>% as.numeric
data$feeling.dem  = zero.one(data$feeling.dem) %>% as.numeric

tmp_dat = data[,c("party3", "authoritarianism", 
                 "female", "age", "college", "income",
                 "jewish", "catholic", "other", "year")] %>% na.omit() %>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism) 




In [ ]:
mod <- brm(party3 ~ female + age + college + income + jewish + 
                  catholic + other + authoritarianism + authoritarianism_2 + 
                  authoritarianism:college + authoritarianism_2:college + 
                 (1+authoritarianism + authoritarianism_2|year), 
                  data = tmp_dat, 
                  family = "categorical",
                  chains = 2, 
                  cores = 6, 
                  seed = 1234, 
                  iter = 1200) 

In [ ]:
main <- brm(party3 ~ female + age + college + income + jewish + 
                  catholic + other + authoritarianism + authoritarianism_2 + 
                 (1+authoritarianism + authoritarianism_2|year), 
                  data = tmp_dat, 
                  family = "categorical",
                  chains = 2, 
                  cores = 6, 
                  seed = 1234, 
                  iter = 1200) 
party_model = list(mod, main)
save(party_model, file = "party_model.rda")

## Affective Polarization Models

The final component in chapter 6 is the affective polarization analysis -- presented as bar charts. Below I estimate the effects for each variable. I did attempt to bind these into a single multivariate regression model (with multiple DVs). This is accomplished using the mvbind command. It took forever to run, so I just did it separately. Finally, I estimate the influence of education (on the difference score).

In [3]:
data$party3      = recode(data$pid*6 + 1, `1` = 1, `2` = 1, `3` = 2 ,`4` = 2, `5` = 2, `6` = 3, `7` = 3) %>% suppressWarnings()
data$republican  = recode(data$pid*6 + 1, `1` = 0, `2` = 0, `3` = 0 ,`4` = 0, `5` = 0, `6` = 1, `7` = 1) %>% suppressWarnings()
data$democrat    = recode(data$pid*6 + 1, `1` = 1, `2` = 1, `3` = 0 ,`4` = 0, `5` = 0, `6` = 0, `7` = 0) %>% suppressWarnings()
data$independent = recode(data$pid*6 + 1, `1` = 0, `2` = 0, `3` = 1 ,`4` = 1, `5` = 1, `6` = 0, `7` = 0) %>% suppressWarnings()
data$feeling.repc = zero.one(data$feeling.repc) %>% as.numeric
data$feeling.demc = zero.one(data$feeling.demc) %>% as.numeric
data$feeling.rep  = zero.one(data$feeling.rep) %>% as.numeric
data$feeling.dem  = zero.one(data$feeling.dem) %>% as.numeric

tmp_dat = data[,c("party3", "authoritarianism", 
                 "female", "age", "college", "income",
                 "jewish", "catholic", "other", "year")] %>% na.omit() %>% 
                 mutate(authoritarianism_2 = authoritarianism*authoritarianism) 

tmp_dat = data[,c("feeling.repc", "feeling.demc", "feeling.rep", "feeling.dem",
                 "authoritarianism", "authoritarianism_2",
                 "female", "age", "college", "income",
                 "jewish", "catholic", "other", "year")] %>% na.omit()


In [ ]:
library(brms)
demc  <- brm(feeling.demc~ female + age + college + income + jewish + 
                  catholic + other + authoritarianism + authoritarianism_2 + 
                  (1+ authoritarianism|year),
                  data = tmp_dat, 
                  family = "gaussian",
                  chains = 2, 
                  cores = 6, 
                  seed = 1234, 
                  iter = 1200) 


In [ ]:
dems <- brm(feeling.dem~ female + age + college + income + jewish + 
                  catholic + other + authoritarianism + authoritarianism_2 + 
                  (1+ authoritarianism|year),
                  data = tmp_dat, 
                  family = "gaussian",
                  chains = 2, 
                  cores = 6, 
                  seed = 1234, 
                  iter = 1200)  


In [ ]:

reps <- brm(feeling.rep~ female + age + college + income + jewish + 
                  catholic + other + authoritarianism + authoritarianism_2 + 
                  (1+ authoritarianism+ authoritarianism_2|year),
                  data = tmp_dat, 
                  family = "gaussian",
                   chains = 2, 
                  cores = 6, 
                  seed = 1234, 
                  iter = 1200)  


In [ ]:

repc <- brm(feeling.repc~ female + age + college + income + jewish + 
                  catholic + other + authoritarianism + authoritarianism_2 + 
                  (1+ authoritarianism+ authoritarianism_2|year), 
                  data = tmp_dat, 
                  family = "gaussian",
                   chains = 2, 
                  cores = 6, 
                  seed = 1234, 
                  iter = 1200) 
affect = list(dems, reps, demc, repc)
save(affect, file = "affect.rda")

In [ ]:

extremity <- brm(I(feeling.rep - feeling.dem)~ female + age + college + income + jewish + 
                  catholic + other + authoritarianism + authoritarianism_2 + 
                  authoritarianism:college + authoritarianism_2:college + 
                  (1+ authoritarianism+ authoritarianism_2|year),
                  data = tmp_dat, 
                  family = "gaussian",
                   chains = 2, 
                  cores = 6, 
                  seed = 1234, 
                  iter = 1200)  
affect = list(dems, reps, demc, repc, extremity)
save(affect, file = "affect.rda")

In [ ]:

extremity2 <- brm(I(feeling.repc - feeling.demc)~ female + age + college + income + jewish + 
                  catholic + other + authoritarianism + authoritarianism_2 + 
                  authoritarianism:college + authoritarianism_2:college + 
                  (1+ authoritarianism+ authoritarianism_2|year),
                  data = tmp_dat, 
                  family = "gaussian",
                   chains = 2, 
                  cores = 6, 
                  seed = 1234, 
                  iter = 1200)
save(extremity2, file = "affect2.rda")

# Indirect and Direct Effects

The quadratic term, coupled with the pid x authoritarianism interaction makes this a particularly complicated model. I wasn't exactly sure how to do this with existing packages. I found this paper which uses something quite similar to what I had proposed in an earlier draft. It's really complex. 

In [ ]:
##Load dependencies, packages, data, etc ##
# More transformations and data recodes.....
rm(list = ls())
library(brms)
library(tidyverse)
library(ggplot2)
library(tictoc)
library(modelr)
library(tidybayes)
library(dplyr)
library(cowplot)
#### Some helper functions
source("/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/configurations/configurations.r")
data_location = "/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/clean_data/"
setwd("/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/Chapters/Chapter8/Chapter6")
#### Load cleaned data
load("/Users/chrisweber/Desktop/Authoritarianism_V2/Authoritarianism_V2/clean_data/pooled.auth.rda")  ### Just work from this data; everything should be here, recoded.
##### persistent ggtheme
ggtheme = theme(
        plot.title=element_text(face="bold",hjust=-.08,vjust=0,colour="#3C3C3C",size=12),
        axis.text.x=element_text(size=9,colour="#535353",face="bold"),
        axis.text.y=element_text(size=9,colour="#535353",face="bold"),
        axis.title.y=element_text(size=11,colour="#535353",face="bold",vjust=1.5),
        axis.ticks=element_blank(),
        panel.grid.major=element_line(colour="#D0D0D0",size=.25),
        panel.background=element_rect(fill="white")) 
####
data$party3      = recode(data$pid*6 + 1, `1` = 1, `2` = 1, `3` = 2 ,`4` = 2, `5` = 2, `6` = 3, `7` = 3) %>% suppressWarnings()
data$republican  = recode(data$pid*6 + 1, `1` = 0, `2` = 0, `3` = 0 ,`4` = 0, `5` = 0, `6` = 1, `7` = 1) %>% suppressWarnings()
data$democrat    = recode(data$pid*6 + 1, `1` = 1, `2` = 1, `3` = 0 ,`4` = 0, `5` = 0, `6` = 0, `7` = 0) %>% suppressWarnings()
data$independent = recode(data$pid*6 + 1, `1` = 0, `2` = 0, `3` = 1 ,`4` = 1, `5` = 1, `6` = 0, `7` = 0) %>% suppressWarnings()
data$feeling.repc = zero.one(data$feeling.repc) %>% as.numeric
data$feeling.demc = zero.one(data$feeling.demc) %>% as.numeric
data$feeling.rep  = zero.one(data$feeling.rep) %>% as.numeric
data$feeling.dem  = zero.one(data$feeling.dem) %>% as.numeric


In [2]:

stan_data = function(data, filter_at = 1990){
                    tmp = data[,c("vote", "authoritarianism", "party3", "feeling.repc", "feeling.demc",
                            "female", "age", "college", "income",
                            "jewish", "catholic", "other", "year")] %>% na.omit() %>% 
                                mutate(authoritarianism_2 = authoritarianism*authoritarianism) %>%  na.omit()%>% 
                                subset(party3!=2)   %>%
                                mutate(party2 = recode(party3, `1` = 0, `3` =1)) 
                    tmp = subset(tmp, year == filter_at)
                    CONTROLS = tmp[,c("female", "age", "college", "income",
                                    "jewish", "catholic", "other")]


                    X = cbind(1, CONTROLS)

                    stan_dat = list(N = nrow(tmp),
                            P = ncol(CONTROLS) + 1,
                            X = X,
                            A  = tmp$authoritarianism,
                            A_2 = tmp$authoritarianism^2,
                            M   = tmp$party2,
                            MxA =  tmp$party2 * tmp$authoritarianism, 
                            MxA_2 = tmp$party2 * tmp$authoritarianism_2,
                            Y    = tmp$vote,
                            alpha_m = rep(0, ncol(X) + 5 ),
                            beta_m =  rep(0, ncol(X) + 2),
                            alpha_vcv = 2.5 * diag(ncol(X) + 5 ),
                            beta_vcv  = 2.5 * diag(ncol(X) + 2 ))

                    return(stan_dat)


} 

In [3]:
moderated_effect = "data{
                int<lower=0> N;                           // number of observations
                int<lower=0> P;                           // number of columns in the control matrix. Call control the matrix of rhs variables minus pid and authoritarianism
                matrix[N, P] X;                           // controls, so excluding authoritariaism and pid
                vector[N]    A;                          // observed authoritarianism, again just extracting stuff from the data matrix
                vector[N]    A_2;                     
                int<lower=0,upper=1> M[N];              
                real MxA[N];              
                real MxA_2[N];           // observed pid
                int<lower=0,upper=1> Y[N];               // voting              
                vector[P + 5]  alpha_m;                   // mean of regression priors.  Again, just declaring according to the 
                vector[P + 2]  beta_m;                    // vector of priors for the mediation equation, PID = .....
                cov_matrix[P + 5] alpha_vcv;             // variance-covariance of regression priors
                cov_matrix[P + 2] beta_vcv;
                    }
        transformed data{
                vector[N] boot_probs = rep_vector(1.0/N, N);     // make vector of 1/N for (classical) bootstrapping. This just assigns an equal probability for bootstrap samples..
                vector[N] vM   = to_vector(M);                  // make vector copy of M
                vector[N] vMxA = to_vector(MxA);                 // make vector copy of MxA
                vector[N] vMxA_2 = to_vector(MxA_2);             // make vector copy of MxA_2
                }
        parameters{
                vector[P + 5] alpha;                         // regression coefficients (outcome model), add 2 because the size of this vector also include pid/auth ints included
                vector[P + 2] beta;                          // regression coefficients (mediator model), add 1 because must also include auth.
                }
        transformed parameters {
                //Save all the parameters in a set of vectors; alpha for the outcome, beta for the mediation
                vector[P] betaZ       = head(beta, P);                 
                vector[P] alphaZ      = head(alpha, P);                
                real betaA            = beta[P + 1];               
                real betaA_2          = beta[P + 2];                 
                real alphaA           = alpha[P + 1];    
                real alphaA_2         = alpha[P + 2];        
                real alphaM           = alpha[P + 3];                
                real alphaMxA         = alpha[P + 4];                             
                real alphaMxA_2       = alpha[P + 5];                             

            }
        model {
            alpha ~ multi_normal(alpha_m, alpha_vcv);                   // priors on causal coefficients weakly informative for binary exposure
            beta  ~ multi_normal(beta_m, beta_vcv);
            // Likelihood in the mediation -- pid model 
            M ~ bernoulli_logit(X * betaZ +  A * betaA + A_2 * betaA_2);                      
           // Likelihood in the outcome -- vote model  
            Y ~ bernoulli_logit(X * alphaZ + A * alphaA +  A_2 * alphaA_2 + vM * alphaM + vMxA * alphaMxA + vMxA_2*alphaMxA_2);
                        }

        generated quantities{
            int row_i;
            real NDE_0 = 0;
            real NDE_1 = 0;

            real NIE_0 = 0;
            real NIE_1 = 0;
            
            vector[N] M_a0;
            vector[N] M_a1;
            
            vector[N] Y_a1_m0;
            vector[N] Y_a0_m0;
            vector[N] Y_a1_m1;
            vector[N] Y_a0_m1;
        for(n in 1:N){
            row_i = categorical_rng(boot_probs);
            M_a0[n] = bernoulli_logit_rng(X[row_i] * betaZ );
            M_a1[n] = bernoulli_logit_rng(X[row_i] * betaZ +  betaA + betaA_2);
            //Use these draws to form various NIE, NDE estimands
            //NDE -- Dem and Rep
            Y_a1_m0[n] = bernoulli_logit_rng(X[row_i] * alphaZ + M_a0[n] * alphaM + alphaA + alphaA_2 + M_a0[n]*1*alphaMxA  + M_a0[n]*1*alphaMxA_2);
            Y_a0_m0[n] = bernoulli_logit_rng(X[row_i] * alphaZ + M_a0[n] * alphaM);
            
            Y_a1_m1[n] = bernoulli_logit_rng(X[row_i] * alphaZ + M_a1[n] * alphaM + alphaA + alphaA_2 + M_a1[n]*1*alphaMxA  + M_a1[n]*1*alphaMxA_2);
            Y_a0_m1[n] = bernoulli_logit_rng(X[row_i] * alphaZ + M_a1[n] * alphaM);
        NDE_0 = NDE_0 + (Y_a1_m0[n] - Y_a0_m0[n])/N; 
        NDE_1 = NDE_1 + (Y_a1_m1[n] - Y_a0_m1[n])/N; 
        NIE_0 = NIE_0 + (Y_a0_m1[n] - Y_a0_m0[n])/N; 
        NIE_1 = NIE_1 + (Y_a1_m1[n] - Y_a1_m0[n])/N; 
    }
}
         
"

In [4]:
# Feeling Thermometer

moderated_effect_linear = "data{
                int<lower=0> N;                           // number of observations
                int<lower=0> P;                           // number of columns in the control matrix. Call control the matrix of rhs variables minus pid and authoritarianism
                matrix[N, P] X;                           // controls, so excluding authoritariaism and pid
                vector[N]    A;                          // observed authoritarianism, again just extracting stuff from the data matrix
                vector[N]    A_2;                     
                int<lower=0,upper=1> M[N];              
                real MxA[N];              
                real MxA_2[N];           // observed pid
                real Y[N];               // ft            
                vector[P + 5]  alpha_m;                   // mean of regression priors.  Again, just declaring according to the 
                vector[P + 2]  beta_m;                    // vector of priors for the mediation equation, PID = .....
                cov_matrix[P + 5] alpha_vcv;             // variance-covariance of regression priors
                cov_matrix[P + 2] beta_vcv;
                    }
        transformed data{
                vector[N] boot_probs = rep_vector(1.0/N, N);     // make vector of 1/N for (classical) bootstrapping. This just assigns an equal probability for bootstrap samples..
                vector[N] vM   = to_vector(M);                  // make vector copy of M
                vector[N] vMxA = to_vector(MxA);                 // make vector copy of MxA
                vector[N] vMxA_2 = to_vector(MxA_2);             // make vector copy of MxA_2
                }
        parameters{
                vector[P + 5] alpha;                         // regression coefficients (outcome model), add 2 because the size of this vector also include pid/auth ints included
                vector[P + 2] beta;                          // regression coefficients (mediator model), add 1 because must also include auth.
                real<lower=0>sigma;
                }
        transformed parameters {
                //Save all the parameters in a set of vectors; alpha for the outcome, beta for the mediation
                vector[P] betaZ       = head(beta, P);                 
                vector[P] alphaZ      = head(alpha, P);                
                real betaA            = beta[P + 1];               
                real betaA_2          = beta[P + 2];                 
                real alphaA           = alpha[P + 1];    
                real alphaA_2         = alpha[P + 2];        
                real alphaM           = alpha[P + 3];                
                real alphaMxA         = alpha[P + 4];                             
                real alphaMxA_2       = alpha[P + 5];                             

            }
        model {
            alpha ~ multi_normal(alpha_m, alpha_vcv);                   // priors on causal coefficients weakly informative for binary exposure
            beta  ~ multi_normal(beta_m, beta_vcv);
            // Likelihood in the mediation -- pid model 
            M ~ bernoulli_logit(X * betaZ +  A * betaA + A_2 * betaA_2);                      
           // Likelihood in the outcome -- vote model  
            Y ~ normal(X * alphaZ + A * alphaA +  A_2 * alphaA_2 + vM * alphaM + vMxA * alphaMxA + vMxA_2*alphaMxA_2, sigma);
                        }

        generated quantities{
            int row_i;
            real NDE_0 = 0;
            real NDE_1 = 0;

            real NIE_0 = 0;
            real NIE_1 = 0;
            
            vector[N] M_a0;
            vector[N] M_a1;
            
            vector[N] Y_a1_m0;
            vector[N] Y_a0_m0;
            vector[N] Y_a1_m1;
            vector[N] Y_a0_m1;
        for(n in 1:N){
            row_i = categorical_rng(boot_probs);
            M_a0[n] = bernoulli_logit_rng(X[row_i] * betaZ );
            M_a1[n] = bernoulli_logit_rng(X[row_i] * betaZ +  betaA + betaA_2);
            //Use these draws to form various NIE, NDE estimands
            //NDE -- Dem and Rep
            Y_a1_m0[n] = normal_rng(X[row_i] * alphaZ + M_a0[n] * alphaM + alphaA + alphaA_2 + M_a0[n]*1*alphaMxA  + M_a0[n]*1*alphaMxA_2, sigma);
            Y_a0_m0[n] = normal_rng(X[row_i] * alphaZ + M_a0[n] * alphaM, sigma);
            
            Y_a1_m1[n] = normal_rng(X[row_i] * alphaZ + M_a1[n] * alphaM + alphaA + alphaA_2 + M_a1[n]*1*alphaMxA  + M_a1[n]*1*alphaMxA_2, sigma);
            Y_a0_m1[n] = normal_rng(X[row_i] * alphaZ + M_a1[n] * alphaM, sigma);
        NDE_0 = NDE_0 + (Y_a1_m0[n] - Y_a0_m0[n])/N; 
        NDE_1 = NDE_1 + (Y_a1_m1[n] - Y_a0_m1[n])/N; 
        NIE_0 = NIE_0 + (Y_a0_m1[n] - Y_a0_m0[n])/N; 
        NIE_1 = NIE_1 + (Y_a1_m1[n] - Y_a1_m0[n])/N; 
    }
}
         
"

In [ ]:
library("rstan")
### Loops through and runs the model on each year. Again, I didn't bother with a multilevel specification here.
for(yy in c(1992, 2000, 2004, 2008, 2012, 2016, 2020)){
    assign(paste0("effect", yy), stan(model_code = moderated_effect, data = stan_data(data, filter_at = yy), iter = 1000,
                                                                                chains = 3, 
                                                                                cores = 6, 
                                                                                seed = 1234)%>% summary(pars = c("NIE_0", "NIE_1", "NDE_0", "NDE_1"))
                                                                                ) 

}


In [7]:
plt1 = rbind(
        effect1992$summary,
        effect2000$summary,
        effect2004$summary,
        effect2008$summary,
        effect2012$summary,
        effect2016$summary,
        effect2020$summary) %>% data.frame() 
save(plt1, file = "indirect_vote.rda")

In [ ]:
library(rstan)

stan_data = function(data, filter_at = 1992){
                    tmp = data[,c("vote", "authoritarianism", "party3", "feeling.repc", "feeling.demc",
                            "female", "age", "college", "income",
                            "jewish", "catholic", "other", "year")] %>% na.omit() %>% 
                                mutate(authoritarianism_2 = authoritarianism*authoritarianism) %>%  na.omit()%>% 
                                subset(party3!=2)   %>%
                                mutate(party2 = recode(party3, `1` = 0, `3` =1)) 
                    tmp = subset(tmp, year == filter_at)
                    CONTROLS = tmp[,c("female", "age", "college", "income",
                                    "jewish", "catholic", "other")]


                    X = cbind(1, CONTROLS)

                    stan_dat = list(N = nrow(tmp),
                            P = ncol(CONTROLS) + 1,
                            X = X,
                            A  = tmp$authoritarianism,
                            A_2 = tmp$authoritarianism^2,
                            M   = tmp$party2,
                            MxA =  tmp$party2 * tmp$authoritarianism, 
                            MxA_2 = tmp$party2 * tmp$authoritarianism_2,
                            Y    = zero.one(tmp$feeling.repc - tmp$feeling.demc),
                            alpha_m = rep(0, ncol(X) + 5 ),
                            beta_m =  rep(0, ncol(X) + 2),
                            alpha_vcv = 2.5 * diag(ncol(X) + 5 ),
                            beta_vcv  = 2.5 * diag(ncol(X) + 2 ))

                    return(stan_dat)


} 
for(yy in c(1992, 2000, 2004, 2008, 2012, 2016, 2020)){
    assign(paste0("effect", yy), stan(model_code = moderated_effect_linear, data = stan_data(data, filter_at = yy), iter = 1000,
                                                                                chains = 3, 
                                                                                cores = 6, 
                                                                                seed = 1234)%>% summary(pars = c("NIE_0", "NIE_1", "NDE_0", "NDE_1"))
                                                                                ) 

}
plt2 = rbind(
        effect1992$summary,
        effect2000$summary,
        effect2004$summary,
        effect2008$summary,
        effect2012$summary,
        effect2016$summary,
        effect2020$summary) %>% data.frame() 


In [13]:
a = list(plt1, plt2)
save(a, file = "indirect.rda")